In [81]:
# -*- coding: utf-8 -*-
# @Time    : 2022/3/30 20:43
# @Author  : huangkai
# @File    : search_content.py


import sys

sys.path.append('../recall/nlp4es8/ir/')
sys.path.append('../utils')

from es_config import Config


# from utils.logger_config import base_logger
# from utils.args import FLAGS
# from ir.index_hnsw import Index


class Search(object):
    def __init__(self, config, index_name):
        # base_logger.info("Searching ...")
        self.config = config
        self.es = self.config.es

        self.index_name = index_name

    def searchAnswer(self, question, ):
        body = {
            "query": {
                "multi_match": {
                    "query": question,
                    "fields": ["idx_document"],  # 在question字段中匹配查询
                    "type": "most_fields",
                }
            }
        }

        # es相关配置

        res = self.es.search(index=self.index_name, body=body, request_timeout=30, size=self.config.top_n)

        topn = res['hits']['hits']

        result = []
        for data in topn:
            result.append(
                (
                    data['_source']['index'],
                    data['_source']['idx_document'],
                    data['_source']['ori_document']
                )
            )
        return result


In [82]:
import pandas as pd
import json

index_file_content_config = Config()
index_file_content_config.index_name = "kbqa"
index_file_content_config.top_n=1
search_name = Search(index_file_content_config, "kbqa")



In [4]:
while True:
    query = input("please input")
    result = search_name.searchAnswer(query)
    print(result[0])
    for data in result:
        print("question:%s  question_id:%s  " % (data[0], data[1]))

please input足球运动员莫侯斯所在的城市是哪个
('6068036', '莫侯斯 ||| 运动项目 ||| 足球')
question:6068036  question_id:莫侯斯 ||| 运动项目 ||| 足球  
question:6068038  question_id:莫侯斯 ||| 所在城市 ||| 米盧斯  
question:7483668  question_id:侯斯特 ||| 职业 ||| 运动员 足球  
question:26744282  question_id:拉莫斯(洪都拉斯足球运动员) ||| 运动项目 ||| 足球  
question:1070692  question_id:莫里斯(足球运动员) ||| 别名 ||| 莫里斯  
question:32913968  question_id:斯莫德斯 ||| 职业 ||| 运动员 足球  
question:10205418  question_id:纳塞夫·莫里斯(足球运动员) ||| 运动项目 ||| 足球  
question:8946580  question_id:霍莫斯 ||| 职业 ||| 运动员 足球  
question:9088935  question_id:迪莫斯 ||| 职业 ||| 运动员 足球  
question:40998095  question_id:夏莫斯 ||| 职业 ||| 运动员 足球  
question:19996452  question_id:斯莫拉 ||| 职业 ||| 运动员 足球  
question:39176369  question_id:巴莫斯 ||| 职业 ||| 运动员 足球  
question:41182628  question_id:拉莫斯 ||| 职业 ||| 足球运动员  
question:16616359  question_id:莫斯达 ||| 职业 ||| 运动员 足球  
question:25987545  question_id:阿斯莫 ||| 职业 ||| 运动员 足球  
question:23974383  question_id:莫伊斯 ||| 职业 ||| 运动员 足球  
question:25779348  question_id:苏莫斯 ||| 职业 |

KeyboardInterrupt: Interrupted by user

In [98]:
import json
data=json.loads(open("es_llm_table_tuple_top20_quchong.json",encoding="utf8").read())

In [23]:
unit

{'id': 'TE001643',
 'question': '“互联网上充斥着各种娱乐信息，今天我意外地发现了一个音乐乐队——冷血动物乐队，对于它来说，我完全是个外行，所以我想了解更多关于这个乐队的信息，例如它的中文名、成立时间、曲目、唱片公司还有他们的成员。”',
 'attribute': ['冷血动物乐队 ||| 中文名 ||| 冷血动物乐队',
  '冷血动物乐队 ||| 成立时间 ||| 1994年',
  '冷血动物乐队 ||| 音乐类型 ||| 摇滚',
  '冷血动物乐队 ||| 类型 ||| 地下摇滚劲旅',
  '冷血动物乐队 ||| 组合形式 ||| 三人组合',
  '冷血动物乐队 ||| 唱片公司 ||| 骏驎唱片',
  '冷血动物乐队 ||| 成员 ||| 谢天笑',
  '李明',
  '梁旭',
  '']}

In [91]:
from tqdm import tqdm
count=0
for unit in data[:-13]:
    temp_attr=[]
    for attr in unit["attribute"]:
        if "没有找到" not in attr:
            ss=attr.split("|||")
            if len(ss)>2:
                result = search_name.searchAnswer("|||".join(ss[:-1]))[0][2]
                if result!=attr:
                    count+=1
                    print(result,"===",attr)

Amy Klinprathum ||| 暱称 ||| Amy === Amy Klinprathum ||| 外文名 ||| Amy Klinprathum;Amy Klinprathum ||| 别名 ||| Amy Klinprathum;Amy Klinprathum ||| 昵称 ||| Amy;Amy Klinprathum ||| 出生地 ||| 美国休斯敦
Amy Klinprathum ||| 配偶 ||| Cee<Siwat Chotechaicharin> === Amy Klinprathum ||| 配偶 ||| Cee<Siwat Chotechaicharin
巴托洛米奥 ||| 悬赏金 ||| 1亿5000万贝里 === 巴托洛米奥 ||| 悬赏金 ||| 1亿5000万贝里(漫画706提到)
哈塞克 ||| 人口（2007年） ||| 81,809 === 哈塞克 ||| 人口(2007年) ||| 81,809
干毛毛 ||| 别 名 ||| 干毛毛 === 干毛毛 ||| 别名 ||| 干毛毛
干毛毛 ||| 别 名 ||| 干毛毛 === 干毛毛 ||| 别名 ||| 干毛毛
干毛毛 ||| 别 名 ||| 干毛毛 === 干毛毛 ||| 别名 ||| 干毛毛
德马雷·卡罗尔 ||| 个人资料 ||| 1986年7月27日（28岁） 阿拉巴马州伯明翰 === 德马雷·卡罗尔 ||| 个人资料 ||| 1986年7月27日(28岁) 阿拉巴马州伯明翰
德马雷·卡罗尔 ||| 登录身高 ||| 6英尺8英寸（2.03米） === 德马雷·卡罗尔 ||| 登录身高 ||| 6英尺8英寸(2.03米)
德马雷·卡罗尔 ||| 登录体重 ||| 212英磅（96千克） === 德马雷·卡罗尔 ||| 登录体重 ||| 212英磅(96千克)
柴碧云 ||| 民族 ||| 汉族 === 柴碧云 ||| 爱好 ||| 看电影
奥斯汀 ||| 人口（2006年） ||| 856,254 === 奥斯汀 ||| 人口(2006年) ||| 856,254
胡锦涛 ||| 副主席 ||| 曾庆红（2003-2008年） 习近平（2008-2013年） === 胡锦涛 ||| 副主席 ||| 曾庆红(2003-2008年) 习近平(2008-2013

宫村优子 ||| 经纪公司 ||| Japan Action Enterprise === 宫村优子 ||| 经纪公司 ||| 声优活动
宫村优子 ||| 代表作品 ||| 《新世纪福音战士》明日香、《名侦探柯南》远山和叶 === 宫村优子 ||| 代表作品 ||| 《》明日香，《名侦探柯南》远山和叶
宫村优子 ||| 歌手活动 ||| 1995年——1999年 === 宫村优子 ||| 歌手活动 ||| 2007年——今
宫村优子 ||| 出生 ||| 1972年12月4日（42岁）  日本兵库县神户市 === 宫村优子 ||| 出生 ||| 1972年12月4日(42岁) 日本兵库县神户市
宫村优子 ||| 出生 ||| 1972年12月4日（42岁）  日本兵库县神户市 === 宫村优子 ||| 出生 ||| 1972年12月4日(42岁) 日本兵库县神户市
凯瑟琳·泽塔-琼斯 ||| 凯瑟琳·泽塔-琼斯 ||| 泽塔-琼斯，2005年2月 === 凯瑟琳·泽塔-琼斯 ||| 别名 ||| 凯瑟琳·泽塔-琼斯
凯瑟琳·泽塔-琼斯 ||| 凯瑟琳·泽塔-琼斯 ||| 泽塔-琼斯，2005年2月 === 凯瑟琳·泽塔-琼斯 ||| 英文名 ||| Catherine Zeta-Jones
凯瑟琳·泽塔-琼斯 ||| 凯瑟琳·泽塔-琼斯 ||| 泽塔-琼斯，2005年2月 === 凯瑟琳·泽塔-琼斯 ||| 出生 ||| 1969年9月25日(45岁) 英国 威尔士 斯旺西
凯瑟琳·泽塔-琼斯 ||| 凯瑟琳·泽塔-琼斯 ||| 泽塔-琼斯，2005年2月 === 凯瑟琳·泽塔-琼斯 ||| 国籍 ||| 英国
凯瑟琳·泽塔-琼斯 ||| 凯瑟琳·泽塔-琼斯 ||| 泽塔-琼斯，2005年2月 === 凯瑟琳·泽塔-琼斯 ||| 职业 ||| 演员
凯瑟琳·泽塔-琼斯 ||| 代表作品 ||| 芝加哥 === 凯瑟琳·泽塔-琼斯 ||| 代表作品 ||| 泽塔-琼斯，2005年2月
凯瑟琳·泽塔·琼斯 ||| 体重 ||| 53 公斤 === 凯瑟琳·泽塔-琼斯 ||| 体重 ||| 53kg
凯瑟琳·泽塔-琼斯 ||| 凯瑟琳·泽塔-琼斯 ||| 泽塔-琼斯，2005年2月 === 凯瑟琳·泽塔-琼斯 ||| 出生 ||| 1969年9月25日(45岁) 英国 威

战前女神 ||| 圈中好友 ||| Jim Brickman，Linda Davis === 战前女神 ||| 圈中好友 ||| Jim Brickman, Linda Davis
许德珩 ||| 逝世 ||| 1990年2月8日  中华人民共和国 === 许德珩 ||| 逝世 ||| 1990年2月8日 中华人民共和国
许德珩 ||| 逝世 ||| 1990年2月8日  中华人民共和国 === 许德珩 ||| 逝世 ||| 1990年2月8日 中华人民共和国
许德珩 ||| 逝世 ||| 1990年2月8日  中华人民共和国 === 许德珩 ||| 逝世 ||| 1990年2月8日 中华人民共和国
许德珩 ||| 逝世 ||| 1990年2月8日  中华人民共和国 === 许德珩 ||| 逝世 ||| 1990年2月8日 中华人民共和国
许合子 ||| 籍贯 ||| 吉州永新县（今江西吉安永新县） === 许合子 ||| 籍贯 ||| 吉州永新县(今江西吉安永新县)
上贺茂润 ||| 能力（招术） ||| 演技高超 === 上贺茂润 ||| 能力(招术) ||| 演技高超
尚圆王 ||| 嫡王女 ||| 闻得大君加那志（童名音智殿茂金，初代闻得大君，号月清） === 尚圆王 ||| 嫡王女 ||| 闻得大君加那志(童名音智殿茂金，初代闻得大君，号月清)
鸠摩智 ||| 职 业 ||| 吐蕃国师 === 鸠摩智 ||| 职业 ||| 吐蕃国师
鸠摩智 ||| 信仰 ||| 佛教（藏传） === 鸠摩智 ||| 信仰 ||| 藏传佛教(藏传)
鸠摩智 ||| 好友 ||| 慕容博 === 鸠摩智 ||| 朋友 ||| 慕容博
鸠摩智 ||| 绝技 ||| 控鹤功 宁玛派： 火焰刀 少林派： 拈花指 多罗叶指 无相劫指 去烦恼指 大智无定指 龙爪功 擒龙手 托钵掌 如影随形腿 袖里乾坤 寂灭抓 因陀罗抓 燃木刀法 摩诃指 般若掌 大金刚拳 === 鸠摩智 ||| 技能 ||| 控鹤功 宁玛派： 火焰刀 少林派： 拈花指 多罗叶指 无相劫指 去烦恼指 大智无定指 龙爪功 擒龙手 托钵掌 如影随形腿 袖里乾坤 寂灭抓 因陀罗抓 燃木刀法 摩诃指 般若掌 大金刚拳
安吉拉·布朗 ||| 职业 ||| 虐杀天使（女性），白执事（男性） === 安吉拉·布朗 ||| 职业 |||

南希·韦克 ||| 部队 ||| SOE F Section networks === 南希·韦克 ||| 军队 ||| SOE F Section networks
南希·韦克 ||| 去世 ||| 2011年8月7日（98岁） 英国伦敦 === 南希·韦克 ||| 去世 ||| 2011年8月7日(98岁) 英国伦敦
片桐彩子 ||| 所属学生社团 ||| 美术社或管乐社（超级任天堂版改为轻音乐社） === 片桐彩子 ||| 所属学生社团 ||| 美术社或管乐社(超级任天堂版改为轻音乐社)
恩维尔·霍查 ||| 逝世 ||| 1985年4月11日（76岁） 阿尔巴尼亚地拉那 === 恩维尔·霍查 ||| 逝世 ||| 1985年4月11日(76岁) 阿尔巴尼亚地拉那
赤木晴子 ||| 赤木晴子 ||| 赤木 晴子（あかぎ はるこ） （Akagi Haruko） === 赤木晴子 ||| 别名 ||| 赤木晴子
赤木晴子 ||| 赤木晴子 ||| 赤木 晴子（あかぎ はるこ） （Akagi Haruko） === 赤木晴子 ||| 中文名 ||| 赤木晴子
赤木晴子 ||| 赤木晴子 ||| 赤木 晴子（あかぎ はるこ） （Akagi Haruko） === 赤木晴子 ||| 外文名 ||| Akagi Haruko
赤木晴子 ||| 国 籍 ||| 日本 === 赤木晴子 ||| 国籍 ||| 日本
赤木晴子 ||| 出生地 ||| 日本 === 赤木晴子 ||| 出生地 ||| 不详
赤木晴子 ||| 班 级 ||| 1年3班 === 赤木晴子 ||| 班级 ||| 1年3班
赤木晴子 ||| 职业 ||| 学生 === 赤木晴子 ||| 职业 ||| 学生，湘北篮球部经理
赤木晴子 ||| 班 级 ||| 1年3班 === 赤木晴子 ||| 班级 ||| 1年3班
萧百佑 ||| 年龄 ||| 50 === 萧百佑 ||| 年 龄 ||| 47
萧百佑 ||| 专 业 ||| 国际金融专业 === 萧百佑 ||| 专业 ||| 国际金融专业
萧百佑 ||| 出生日期 ||| 1964年 === 萧百佑 ||| 出生地 ||| 广东
萧百佑 ||| 年龄 ||| 50 === 萧百佑 ||| 年 龄 ||| 50
萧百佑 ||| 专 业 ||| 国际金融专业 

名侦探柯南 ||| 话数 ||| 916话（2015年1月现在） === 名侦探柯南 ||| 话数 ||| 916话(2015年1月现在)
阿尔马斯 ||| 人口（2007） ||| 7,488 === 阿尔马斯 ||| 人口(2007) ||| 7,488
高俊熙 ||| 兴趣 ||| 音乐欣赏、电影欣赏、唱歌 === 高俊熙 ||| 兴趣 ||| 唱歌、跳舞
古典诗歌名句辞典 ||| 装帧 ||| :精装 === 古典诗歌名句辞典 ||| 装帧 ||| ：精装
杰西卡·布尔西亚 ||| 出生日期 ||| April 11，1983 === 杰西卡·布尔西亚 ||| 出生日期 ||| April 11,1983
奈芙蒂斯 ||| 奈芙蒂斯 ||| 奈芙蒂斯 === 奈芙蒂斯 ||| 别名 ||| 奈芙蒂斯
奈芙蒂斯 ||| 奈芙蒂斯 ||| 奈芙蒂斯 === 奈芙蒂斯 ||| 中文名 ||| 奈芙蒂斯
奈芙蒂斯 ||| 奈芙蒂斯 ||| 奈芙蒂斯 === 奈芙蒂斯 ||| 外文名 ||| Nephthys
弗朗克·韦尔切克 ||| 出生 ||| Frank Anthony Wilczek 1951年5月15日（63岁）  美国纽约州Mineola（英语：Mineola, New York） === 弗朗克·韦尔切克 ||| 出生 ||| Frank Anthony Wilczek 1951年5月15日(63岁) 美国纽约州Mineola(英语：Mineola, New York) 弗朗克·韦尔切克 ||| 出生地 ||| 纽约州米里奥拉 弗朗克·韦尔切克 ||| 职业 ||| 物理学家 弗朗克·韦尔切克 ||| 毕业院校 ||| 芝加哥大学(B.S.) 普林斯顿大学 (M.A.、Ph.D.) 弗朗克·韦尔切克 ||| 研究领域 ||| 物理学 数学 弗朗克·韦尔切克 ||| 著名成就 ||| Asymptotic Freedom Quantum chromodynamics Quantum Statistics 弗朗克·韦尔切克 ||| 英文名 ||| Frank Wilczek 弗朗克·韦尔切克 ||| 出生年月 ||| 1951年5月15日
琦善 ||| 出生 ||| 乾隆五十年十二月十九日 （1786年1月18日）[1] 京师 === 琦善 ||| 出生 |

洪朝丰 ||| 代表作 ||| 《壹号皇庭》,《射雕英雄传》 ，《黄埔倾情》等 === 洪朝丰 ||| 代表作 ||| 《壹号皇庭》,《射雕英雄传》,《黄埔倾情》等
师东兵 ||| 职 业 ||| 作家 === 师东兵 ||| 职业 ||| 作家
陈发枝 ||| 职业俱乐部* ||| 出场（入球） === 陈发枝 ||| 职业俱乐部* ||| 出场(入球)
陈发枝 ||| 职业俱乐部* ||| 出场（入球） === 陈发枝 ||| 职业俱乐部 ||| 已退役
陈咏谦 ||| 毕业院校 ||| 九龙华仁书院， === 陈咏谦 ||| 毕业院校 ||| 九龙华仁书院
芭儿·拉法莉 ||| 衣服尺寸 ||| 36（欧盟）；6（美国） === 芭儿·拉法莉 ||| 衣服尺寸 ||| 36(欧盟);6(美国)
吴桐 ||| 职业 ||| 演员 模特 === 吴桐 ||| 职业 ||| 作家
吴桐 ||| 宗教信仰 ||| 共产主义 === 吴桐 ||| 宗教 ||| 共产主义
吴桐 ||| 职业 ||| 演员 模特 === 吴桐 ||| 职 业 ||| 演员 模特
阿曼德 ||| 原名 ||| Andrei（安德烈） === 阿曼德 ||| 原名 ||| Andrei(安德烈)
傅履仁 ||| 亲属 ||| 父傅泾波、母刘倬汉 姊傅暖泠、傅铎若、傅海澜 子Jarrett Fugh、女Justina Fugh === 傅履仁 ||| 亲属 ||| 父傅泾波、母刘倬汉 姊傅暖泠、傅铎若、傅海澜 子Jarrett Fugh、女 Justina Fugh
元昭宗 ||| 登基 ||| 元惠宗至正三十年农历四月二十八日 1370年5月23日 应昌（今内蒙古自治区赤峰市克什克腾旗西北达里诺尔西南的达日罕乌拉苏木） === 元昭宗 ||| 登基 ||| 元惠宗至正三十年农历四月二十八日 1370年5月23日 应昌(今内蒙古自治区赤峰市克什克腾旗西北达里诺尔西南的达日罕乌拉苏木)
艾丽斯·芒罗 ||| 别名 ||| 艾丽斯·芒罗 === 艾丽斯·芒罗 ||| 别名 ||| 艾丽丝·门罗
玛丽·麦卡利斯 ||| 个人资料 ||| 1951年6月27日（63岁） 北爱尔兰贝尔法斯特 === 玛丽·麦卡利斯 ||| 个人资料 ||| 1951年6月27日(63岁) 北爱尔兰贝尔法斯特
安与骑兵 |

卡里卡里 ||| 身高 ||| 187.0CM === 卡里 ||| 身高 ||| 154厘米
卡里卡里 ||| 身高 ||| 187.0CM === 卡里 ||| 身高 ||| 153厘米
卡里卡里 ||| 身高 ||| 187.0CM === 卡里 ||| 身高 ||| 152厘米
卡里卡里 ||| 身高 ||| 187.0CM === 卡里 ||| 身高 ||| 151厘米
卡里卡里 ||| 身高 ||| 187.0CM === 卡里 ||| 身高 ||| 150厘米
卡里卡里 ||| 身高 ||| 187.0CM === 卡里 ||| 身高 ||| 149厘米
卡里卡里 ||| 身高 ||| 187.0CM === 卡里 ||| 身高 ||| 148厘米
卡里卡里 ||| 身高 ||| 187.0CM === 卡里 ||| 身高 ||| 147厘米
卡里卡里 ||| 身高 ||| 187.0CM === 卡里 ||| 身高 ||| 146厘米
卡里卡里 ||| 身高 ||| 187.0CM === 卡里 ||| 身高 ||| 145厘米
卡里卡里 ||| 身高 ||| 187.0CM === 卡里 ||| 身高 ||| 144厘米
卡里卡里 ||| 身高 ||| 187.0CM === 卡里 ||| 身高 ||| 143厘米
卡里卡里 ||| 身高 ||| 187.0CM === 卡里 ||| 身高 ||| 142厘米
卡里卡里 ||| 身高 ||| 187.0CM === 卡里 ||| 身高 ||| 141厘米
卡里卡里 ||| 身高 ||| 187.0CM === 卡里 ||| 身高 ||| 140厘米
卡里卡里 ||| 身高 ||| 187.0CM === 卡里 ||| 身高 ||| 139厘米
卡里卡里 ||| 身高 ||| 187.0CM === 卡里 ||| 身高 ||| 138厘米
卡里卡里 ||| 身高 ||| 187.0CM === 卡里 ||| 身高 ||| 137厘米
卡里卡里 ||| 身高 ||| 187.0CM === 卡里 ||| 身高 ||| 136厘米
卡里卡里 ||| 身高 ||| 187.0CM === 卡里 ||| 身高 ||| 135厘米
卡里卡里 ||| 身高 ||| 187.0CM === 卡里 ||| 身高 ||

In [90]:
unit

{'id': 'TE001277',
 'question': '叶子媚的三围情况如何？',
 'attribute': ['叶子媚 ||| 三围尺寸 ||| 36英寸', '21英寸', '35英寸']}

In [99]:
count=0
for unit in tqdm(data[:-13]):
    temp_attr=[]
    for attr in unit["attribute"]:
        if attr:
            if "没有找到" not in attr:
                if ";" in attr:
                    attr=attr.split(";")
                    for a in attr:
                        a=a.split("|||")
                        if len(a)>2:
                            result = search_name.searchAnswer("|||".join(a[:-1]))[0][2]
                            temp_attr.append(result)
                else:
                    attr=attr.split("|||")
                    if len(attr)>2:
                        result = search_name.searchAnswer("|||".join(attr[:-1]))[0][2]
                        temp_attr.append(result)
            else:
                temp_attr.append(attr)
    unit["attribute"] = temp_attr

100%|██████████████████████████████████████████████████████████████████████████████| 2000/2000 [00:58<00:00, 34.38it/s]


In [107]:
data[-12]

{'id': 'TEDOC002',
 'question': '请描述一个驴友出游受困多日成功获救的写实故事，不少于500字',
 'attribute': ['驴友们在荒野中受困多日，历经千辛万苦，最终成功获救的故事。这次出游是由一群热爱探险的驴友组成的队伍。他们经验丰富，充满热情，享受着在荒野中度过的每一刻。然而，命运却给他们开了一个残酷的玩笑。在穿越一片险峻的山林时，突如其来的山洪将他们困在了一个峡谷中。面对严酷的自然环境，驴友们并未灰心丧气。他们团结一心，互相扶持，决定共同面对这场危机。被困的每一天，他们都有条不紊地进行自救。利用有限的资源，合理分配食物和水，时刻保持警惕。然而，尽管他们努力寻找出路，但始终没有发现任何逃脱的线索。就在他们即将陷入绝望的时候，一名驴友在仔细地搜寻中，发现了一个被隐藏的洞口。他们立刻行动起来，合力挖掘，希望这会是他们的救星。在经过一段艰难的时间后，他们终于挖出了一条通道，成功地找到了出路。就在他们准备松一口气时，却发现当地的村民已经为他们准备了热汤和温暖的床铺。这些村民得知他们受困的消息后，自发组织了救援行动，给他们提供了力所能及的帮助。在那个最漫长的夜晚，他们围坐在篝火旁，分享着村民们的温暖和食物。他们感慨万千，相互倾诉着心声。在这生死攸关的时刻，他们更加珍惜彼此之间的友谊，相互鼓舞着，一定要坚强地活下去。最终，救援队在第二天及时赶到，将他们全部安全救出。获救的驴友们感激涕零，他们知道，这次能够成功获救，离不开他们之间的团结互助，以及那些无私奉献的当地村民。这次历险让他们深刻体会到生命的脆弱与珍贵。他们明白了一个道理：在面对困境时，只有团结一致，勇敢面对，才能战胜一切困难。同时，他们也深知应该珍惜生命中的每一刻，感恩每一个帮助过自己的人。获救后的驴友们决定将这段经历永远铭记在心。他们通过组织各种公益活动和安全培训课程，向更多的人分享自己的经历和教训。他们希望能够帮助更多的人提高安全意识，避免类似的悲剧再次发生。这个故事传遍了四海，成为了一个激励人心的真实故事。它告诉我们，无论何时何地，只要我们团结一心，勇敢面对困境，就一定能够战胜一切挑战。同时，我们也要珍惜生命中的每一刻，关爱他人，回报社会。这个故事成为了无数人心中的力量源泉。让我们明白：生命是脆弱的，但也是坚韧的；友谊是宝贵的，但也是易逝的；困难是可怕的，但也是可以战胜的

In [ ]:
for unit in data[-13:]:
    temp_dic={
        "id":unit["id"],
        "question":unit["question"],
        "attribute":unit["attribute"]
    }
    final_result.append(temp_dic)

In [105]:
data[1111]

{'id': 'TE000761',
 'question': '长泽筑实作为模特是什么类型的？',
 'attribute': ['长泽筑实 ||| 模特类型 ||| 水着']}

In [34]:
unit["attribute"]

['冷血动物乐队 ||| 中文名 ||| 冷血动物乐队',
 '冷血动物乐队 ||| 成立时间 ||| 1994年',
 '冷血动物乐队 ||| 音乐类型 ||| 摇滚',
 '冷血动物乐队 ||| 类型 ||| 地下摇滚劲旅',
 '冷血动物乐队 ||| 组合形式 ||| 三人组合',
 '冷血动物乐队 ||| 唱片公司 ||| 骏驎唱片',
 '冷血动物乐队 ||| 成员 ||| 谢天笑',
 '李明',
 '梁旭',
 '']

In [14]:
count/len(data[:-13])

0.224

In [15]:
attr

'汪小敏 ||| 喜欢的电视 ||| 《吸血鬼日记》'

In [106]:
submission = open("es_llm_table_tuple_top20_quchong_post_process.json", "w", encoding="utf8")

submission.write(json.dumps(data, ensure_ascii=False))

297932

In [45]:
data[2]

{'id': 'TE001070',
 'question': '哪些官职在松平赖桓的身上',
 'attribute': ['松平赖桓 ||| 朝廷官位 ||| 从四位上、讃岐守、侍从']}

In [52]:
st='霸州市新利钢铁有限公司 ||| 别名 ||| 霸州市新利钢铁有限公司'
str(set(st))

"{'钢', '限', '利', '|', ' ', '新', '司', '州', '名', '市', '霸', '别', '公', '有', '铁'}"

In [53]:
import json
data=json.loads(open("es_llm_top20_test_quchong.json",encoding="utf8").read())

In [61]:
final_result=[]
for unit in data[:-13]:
    temp_dic={
        "id":unit["id"],
        "question":unit["question"],
        "attribute":unit["top20"]
    }
    final_result.append(temp_dic)

In [63]:
for unit in data[-13:]:
    temp_dic={
        "id":unit["id"],
        "question":unit["question"],
        "attribute":unit["attribute"]
    }
    final_result.append(temp_dic)

In [59]:
final_result[3]

{'id': 'TE000550',
 'question': '什么是杰基·乔伊纳一克西获得的单人最好成就？',
 'attribute': ['杰基·乔伊纳一克西 ||| 个人最好成绩 ||| 女子七项全能7291分',
  '杰基·乔伊纳一克西 ||| 别名 ||| 杰基·乔伊纳一克西',
  '杰基·乔伊纳一克西 ||| 国籍 ||| 美国',
  '杰基·乔伊纳一克西 ||| 中文名 ||| 杰基·乔伊纳·克西',
  '杰基·乔伊纳一克西 ||| 出生地 ||| 伊利诺伊州',
  '杰基·乔伊纳一克西 ||| 外文名 ||| JACKIEJOYNER－KERSEE',
  '杰基·乔伊纳一克西 ||| 出生日期 ||| 1962年3月3日',
  '杰基·乔伊纳一克西 ||| 运动项目 ||| 七项全能',
  '杰基·乔伊纳一克西 ||| 主要奖项 ||| 第24届奥运会跳远金牌 第24届奥运会七项全能金牌',
  '杰基·乔伊纳一克西 ||| 重要事件 ||| 首位突破女子七项全能“7000分”',
  '乔纳森·杰克逊 ||| 主要成就 ||| 艾美奖',
  '乔伊纳·克西 ||| 性别 ||| 女',
  '乔伊纳·克西 ||| 国籍 ||| 美国',
  '乔伊纳·克西 ||| 别名 ||| 乔伊纳·克西',
  '乔伊纳·克西 ||| 职业 ||| 田径',
  '乔伊纳·克西 ||| 英文名 ||| Joyner Kersee',
  '乔伊纳·克西 ||| 出生地 ||| 美国的伊利诺伊州',
  '乔伊纳·克西 ||| 中文名 ||| 乔伊纳•克西',
  '乔纳森·伊夫 ||| 主要成就 ||| 参与设计苹果产品',
  '乔纳森·盖伊 ||| 主要成就 ||| Flash 最初的设计者']}

In [64]:
submission = open("es_llm_tuple_top20_quchong.json", "w", encoding="utf8")

submission.write(json.dumps(final_result, ensure_ascii=False))

1386413

In [65]:
import json
data=json.loads(open("es_top20_train.json",encoding="utf8").read())

In [73]:
sums,maxs=[],0
for unit in data:
    length=len(unit["attribute"])
    if length>maxs:
        maxs=length

In [74]:
maxs

27